In [ ]:
!pip install mlflow

     |████████████████████████████████| 13.9MB 316kB/s 
     |████████████████████████████████| 153kB 50.3MB/s 
     |████████████████████████████████| 81kB 6.1MB/s 
     |████████████████████████████████| 61kB 6.1MB/s 
     |████████████████████████████████| 1.1MB 49.9MB/s 
     |████████████████████████████████| 337kB 39.0MB/s 
     |████████████████████████████████| 163kB 50.1MB/s 
     |████████████████████████████████| 204kB 50.2MB/s 
     |████████████████████████████████| 81kB 8.5MB/s 
     |████████████████████████████████| 133kB 57.2MB/s 
     |████████████████████████████████| 92kB 8.6MB/s 
     |████████████████████████████████| 2.6MB 49.6MB/s 
     |████████████████████████████████| 71kB 7.2MB/s 
     |████████████████████████████████| 51kB 5.1MB/s 
  Created wheel for prometheus-flask-exporter: filename=prometheus_flask_exporter-0.18.1-cp36-none-any.whl size=17159 sha256=d3aa3ad4967228f8fbd389db0dc44f268fd37c9a266f8c7cb00064175c081264
  Stored in directory: /root/.cache/pi

In [ ]:
!pip install pyngrok


  Created wheel for pyngrok: filename=pyngrok-5.0.0-cp36-none-any.whl size=18781 sha256=34a381d52cae29a528b8f952c8bfca8d3077fd47afcccb5558b809f8645bb279
  Stored in directory: /root/.cache/pip/wheels/95/df/23/af8dde08c3fcdc7b966adcacef48ab29aa3b0b1860df5d2b79
Successfully built pyngrok


In [ ]:
get_ipython().system_raw("mlflow ui --port 5000 &")

In [ ]:
from pyngrok import ngrok


In [ ]:
ngrok.kill()


In [ ]:
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)


In [ ]:
print("MLflow UI ", ngrok_tunnel.public_url)


MLflow UI  https://a555ad5d48fa.ngrok.io


In [ ]:
import numpy as np
import pandas as pd
import mlflow
import mlflow.sklearn

if __name__ == "__main__":

    mlflow.set_experiment(experiment_name="mlflow demo")
    
    training_data = pd.read_csv('https://raw.githubusercontent.com/futurexskill/ml-model-deployment/main/storepurchasedata.csv')
    print("loaded training data")

    training_data.describe()
    mlflow.log_param("training percentage",60)

    mlflow.log_param("dataset shape",training_data.shape)
    
    X = training_data.iloc[:, :-1].values
    y = training_data.iloc[:,-1].values
    
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =.4,random_state=0)
    
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    print("Completed Feature Scaling")
    
    from sklearn.neighbors import KNeighborsClassifier
    # minkowski is for ecledian distance
    classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
    
    # Model training
    classifier.fit(X_train, y_train)
    print("Model trained")
   
    y_pred = classifier.predict(X_test)
    y_prob = classifier.predict_proba(X_test)[:,1]
    
    from sklearn.metrics import confusion_matrix
    
    cm = confusion_matrix(y_test, y_pred)
    
    from sklearn.metrics import accuracy_score
    
    model_accuracy = accuracy_score(y_test,y_pred)
    
    print(model_accuracy)
    
    mlflow.log_metric("accuracy", model_accuracy)
    mlflow.sklearn.log_model(classifier, "model")
    
    

loaded training data
Completed Feature Scaling
Model trained
0.8125


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

import pandas as pd
import numpy as np

import mlflow
import mlflow.pytorch

if __name__ == "__main__":
    
    mlflow.set_experiment(experiment_name="mlflow pytorch demo")
    
    dataset = pd.read_csv('https://raw.githubusercontent.com/futurexskill/ml-model-deployment/main/storepurchasedata_large.csv')
    
    dataset.describe()
    
    dataset.head()
    
    X = dataset.iloc[:, :-1].values
    y = dataset.iloc[:,-1].values
    
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =.20,random_state=0)
    
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    Xtrain_ = torch.from_numpy(X_train).float()
    Xtest_ = torch.from_numpy(X_test).float()
    
    Xtrain_
    
    ytrain_ = torch.from_numpy(y_train)
    ytest_ = torch.from_numpy(y_test)
    
    ytrain_
    
    Xtrain_.shape, ytrain_.shape
    
    Xtest_.shape, ytest_.shape
    
    input_size=2
    output_size=2
    hidden_size=10
    
    class Net(nn.Module):
       def __init__(self):
           super(Net, self).__init__()
           self.fc1 = torch.nn.Linear(input_size, hidden_size)
           self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
           self.fc3 = torch.nn.Linear(hidden_size, output_size)
    
    
       def forward(self, X):
           X = torch.relu((self.fc1(X)))
           X = torch.relu((self.fc2(X)))
           X = self.fc3(X)
    
           return F.log_softmax(X,dim=1)
    
    model = Net()
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    loss_fn = nn.NLLLoss()
    
    epochs = 50
    
    for epoch in range(epochs):
      optimizer.zero_grad()
      Ypred = model(Xtrain_)
      loss = loss_fn(Ypred,  ytrain_)
      loss.backward()
      optimizer.step()
      print('Epoch',epoch, 'loss',loss.item())
    
    mlflow.end_run()
    with mlflow.start_run() as run:
        mlflow.log_param("epochs", 50)
        mlflow.pytorch.log_model(model, "models")
        mlflow.log_metric("loss", loss.item())



Epoch 0 loss 0.6217209100723267
Epoch 1 loss 0.6080432534217834
Epoch 2 loss 0.5946632027626038
Epoch 3 loss 0.5813021063804626
Epoch 4 loss 0.5672823190689087
Epoch 5 loss 0.552306056022644
Epoch 6 loss 0.5361059308052063
Epoch 7 loss 0.5190396904945374
Epoch 8 loss 0.5011383295059204
Epoch 9 loss 0.482537180185318
Epoch 10 loss 0.4638012945652008
Epoch 11 loss 0.4441814124584198
Epoch 12 loss 0.4238114655017853
Epoch 13 loss 0.40323126316070557
Epoch 14 loss 0.38265877962112427
Epoch 15 loss 0.3624744713306427
Epoch 16 loss 0.3428744673728943
Epoch 17 loss 0.3240891695022583
Epoch 18 loss 0.3062521517276764
Epoch 19 loss 0.28950539231300354
Epoch 20 loss 0.2737556993961334
Epoch 21 loss 0.2591138482093811
Epoch 22 loss 0.2456398904323578
Epoch 23 loss 0.23318041861057281
Epoch 24 loss 0.22161489725112915
Epoch 25 loss 0.21087764203548431
Epoch 26 loss 0.20089055597782135
Epoch 27 loss 0.1916988343000412
Epoch 28 loss 0.18325361609458923
Epoch 29 loss 0.17542989552021027
Epoch 30 loss